## Nama : Patrick Jonathan

## NIM : 2440064791

## Video Link : https://www.youtube.com/watch?v=VRvAGi_1QX0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Nomor 1

Import Library yang dibutuhkan untuk nomor 1

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec

Import data yang digunakan untuk menyelesaikan kasus nomor 1

In [ ]:
data01 = pd.read_csv('/content/drive/MyDrive/UAS Text Mining/data_1A.csv')

In [ ]:
data01.head()

,Unnamed: 0,text,label
0,0,The Theory of Everything Review Stephen Hawkin...,Books
1,1,Computer Networks: A Top - Down Approach About...,Books
2,2,Sajani Premium Quality Brown Wooden Coat Hange...,Household
3,3,Bosch Lifestyle MCM3501M 800-Watt Food Process...,Household
4,4,Secret Wish Women's Navy-Blue Towel Bathrobe (...,Household


In [ ]:
data01['text'] = data01['text'].astype(str)

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### a. Pre-processing

Lakukan pre-processing untuk mendapatkan set token yang sudah berada dalam
bentuk dasar sesuai standard tata bahasa. Pada kode dibawah ini adalah fungsi untuk melakukan pre-processing dengan case folding, menghilangkan angka, menghilangkan tanda baca, dan menghilangkan spasi berlebih. Selanjutnya teks akan ditokenisasi menjadi kata-kata. Kemudian, kita akan menghilangkan stop word dalam token tersebut. Terakhir, lakukan lematisasi kata dari token yang diperoleh.

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Case folding
    text = text.lower()

    # Remove number
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove multiple space
    text = re.sub(r'\s+', ' ', text)

    # Tokenisasi teks menjadi kata-kata
    tokens = word_tokenize(text)

    # Hapus stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Lematisasi kata
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

Terapkan fungsi text pre-processing yang sudah dibuat pada kolom text dari data01

In [ ]:
data01['text'] = data01['text'].apply(preprocess_text)
print(data01['text'])

0        [theory, everything, review, stephen, hawking,...
1        [computer, network, top, approach, author, beh...
2        [sajani, premium, quality, brown, wooden, coat...
3        [bosch, lifestyle, mcmm, watt, food, processor...
4        [secret, wish, woman, navyblue, towel, bathrob...
                               ...                        
12601    [lotus, makeup, ecostay, insta, smooth, perfec...
12602    [subtle, art, giving, fck, review, resilience,...
12603    [elevanto, premium, collection, th, sleeve, te...
12604    [wd, passport, tb, portable, external, hard, d...
12605    [storite, foot, mm, male, mm, female, jack, ex...
Name: text, Length: 12606, dtype: object


Berdasarkan output diatas, dapat dilihat bahwa data sudah dibersihkan dan diubah ke dalam bentuk token

### b. Classification Using Machine Learning

Selanjutnya, kita lakukan vektorisasi pada kolom label agar kemudian dapat diklasifikasikan dengan menggunakan algoritma machine learning.

In [ ]:
label_encoder = LabelEncoder()
transformed_label = label_encoder.fit_transform(data01['label'])

Selanjutnya data akan displit ke dalam train dan test dengan proporsi 80% data train dan 20% data test. Kemudian, disini saya menggunakan 2 metode vectorization yaitu CBOW (Continuous Bag-of-Words) dan skip-gram. Saya membangun kedua model tersebut dengan `vector-size` = 100, dimana setiap kata akan direpresentasikan sebagai vektor 100-dimensi. `window` diatur sebagai 5, yang berarti akan diambil 5 kata sebelum dan 5 kata sesudah kata target sebagai konteks. `min_count` diatur sebagai 1, yang berarti semua kata akan diperhitungkan.

Kemudian, lakukan vektorisasi dengan mengubah token ke dalam bentuk vektor CBOW dan skip-gram.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data01['text'], transformed_label, test_size=0.2, random_state=42)

# Membangun model Word2Vec CBOW
w2v_cbow_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, sg=0)

# Membangun model Word2Vec Skip-gram
w2v_skipgram_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, sg=1)

def get_text_vector(text, model):
    vector = np.zeros(model.vector_size)
    count = 0
    for word in text:
        if word in model.wv:
            vector += model.wv[word]
            count += 1
    if count != 0:
        vector /= count
    return vector

# Mengubah teks menjadi vektor fitur dengan Word2Vec CBOW
X_train_vectors_cbow = [get_text_vector(text, w2v_cbow_model) for text in X_train]
X_test_vectors_cbow = [get_text_vector(text, w2v_cbow_model) for text in X_test]

# Mengubah teks menjadi vektor fitur dengan Word2Vec Skip-gram
X_train_vectors_skipgram = [get_text_vector(text, w2v_skipgram_model) for text in X_train]
X_test_vectors_skipgram = [get_text_vector(text, w2v_skipgram_model) for text in X_test]

Hasil vektor tersebut akan dimasukkan ke dalam algoritma machine learning untuk diklasifikasi. Disini saya menggunakan 2 algoritma untuk melakukan klasifikasi yaitu, SVM (Support Vector Machine) dan Random Forest. Disini saya melakukan proses training dan menampilkan hasil akurasi prediksi pada dataset test.

In [ ]:
svm = SVC()
random_forest = RandomForestClassifier()

# SVM dengan Word2Vec CBOW
svm.fit(X_train_vectors_cbow, y_train)
y_pred_svm_cbow = svm.predict(X_test_vectors_cbow)
accuracy_svm_cbow = accuracy_score(y_test, y_pred_svm_cbow)
print("Accuracy (SVM - CBOW):", accuracy_svm_cbow)

# Random Forest dengan Word2Vec CBOW
random_forest.fit(X_train_vectors_cbow, y_train)
y_pred_rf_cbow = random_forest.predict(X_test_vectors_cbow)
accuracy_rf_cbow = accuracy_score(y_test, y_pred_rf_cbow)
print("\nAccuracy (Random Forest - CBOW):", accuracy_rf_cbow)

# SVM dengan Word2Vec Skip-gram
svm.fit(X_train_vectors_skipgram, y_train)
y_pred_svm_skipgram = svm.predict(X_test_vectors_skipgram)
accuracy_svm_skipgram = accuracy_score(y_test, y_pred_svm_skipgram)
print("\nAccuracy (SVM - Skip-gram):", accuracy_svm_skipgram)

# Random Forest dengan Word2Vec Skip-gram
random_forest.fit(X_train_vectors_skipgram, y_train)
y_pred_rf_skipgram = random_forest.predict(X_test_vectors_skipgram)
accuracy_rf_skipgram = accuracy_score(y_test, y_pred_rf_skipgram)
print("\nAccuracy (Random Forest - Skip-gram):", accuracy_rf_skipgram)

Accuracy (SVM - CBOW): 0.9302141157811261

Accuracy (Random Forest - CBOW): 0.9460745440126883

Accuracy (SVM - Skip-gram): 0.9409199048374306

Accuracy (Random Forest - Skip-gram): 0.9480570975416336


Berdasarkan output diatas, kita dapat melihat hasil akurasi dari algoritma yang digunakan.

Jika dikaji dari segi metode vektorisasi, dapat dilihat bahwa metode skip-gram mwmberikan hasil akurasi yang lebih baik daripada metode CBOW. Hal ini dapat dilihat dari akurasi `SVM Skip-gram` (0,94) yang lebih besar dari akurasi `SVM CBOW` (0,93), serta akurasi `Random Forest Skip-gram` (0,948) yang lebih besar dari akurasi `Random Forest CBOW` (0,946).

Jika dikaji dari segi Algoritma Klasifikasi, dapat dilihat bahwa algoritma Random Forest memberikan hasil akurasi yang lebih baik daripada algoritma SVM. Hal ini dapat dilihat dari akurasi `Random Forest CBOW` (0,946) yang lebih besar dari akurasi `SVM CBOW` (0,93), serta akurasi `Random Forest Skip-gram` (0,948) yang lebih besar dari akurasi `SVM Skip-gram` (0,941).

In [ ]:
target_names = np.unique(data01['label'])

# Classification Report - SVM dengan Word2Vec CBOW
print('Classification Report (SVM - Word2Vec CBOW):\n')
print(classification_report(y_test, y_pred_svm_cbow, target_names=target_names))

# Classification Report - SVM dengan Word2Vec Skip-gram
print('Classification Report (SVM - Word2Vec Skip-gram):\n')
print(classification_report(y_test, y_pred_svm_skipgram, target_names=target_names))

# Classification Report - SVM dengan Word2Vec CBOW
print('Classification Report (Random Forest - Word2Vec CBOW):\n')
print(classification_report(y_test, y_pred_rf_cbow, target_names=target_names))

# Classification Report - SVM dengan Word2Vec Skip-gram
print('Classification Report (Random Forest - Word2Vec Skip-gram):\n')
print(classification_report(y_test, y_pred_rf_skipgram, target_names=target_names))

Classification Report (SVM - Word2Vec CBOW):

                        precision    recall  f1-score   support

                 Books       0.96      0.92      0.94       586
Clothing & Accessories       0.93      0.94      0.94       466
           Electronics       0.92      0.90      0.91       506
             Household       0.92      0.95      0.93       964

              accuracy                           0.93      2522
             macro avg       0.93      0.93      0.93      2522
          weighted avg       0.93      0.93      0.93      2522

Classification Report (SVM - Word2Vec Skip-gram):

                        precision    recall  f1-score   support

                 Books       0.95      0.91      0.93       586
Clothing & Accessories       0.94      0.95      0.95       466
           Electronics       0.93      0.92      0.93       506
             Household       0.94      0.96      0.95       964

              accuracy                           0.94      2522
  

Berdasarkan hasil Classification Report diatas, dapat dilihat bahwa semua kategori baik Books, Clothing & Accessories, Electronics, dan Household dapat diklasifikasikan dengan baik dimana semua akurasinya berada diatas 90%. Namun dapat dilihat berdasarkan output semua algoritma diatas, kategori Books memiliki akurasi yang paling tinggi diantara semua kategori lainnya. Hal ini kemungkinan dikarenakan kategori `Books` memiliki deskripsi produk yang lebih menonjol atau lebih unik sehingga mudah diklasifikasikan sebagai kategori `Books`.

## Nomor 2

In [ ]:
!pip install transformers

Import Library yang dibutuhkan untuk nomor 2

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer
import pandas as pd
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

Import data yang digunakan untuk menyelesaikan kasus nomor 2. Dalam hal ini data yang digunakan sama dengan data yang dipakai pada nomor 1

In [ ]:
data02 = pd.read_csv('/content/drive/MyDrive/UAS Text Mining/data_1A.csv')

In [ ]:
data02.head()

,Unnamed: 0,text,label
0,0,The Theory of Everything Review Stephen Hawkin...,Books
1,1,Computer Networks: A Top - Down Approach About...,Books
2,2,Sajani Premium Quality Brown Wooden Coat Hange...,Household
3,3,Bosch Lifestyle MCM3501M 800-Watt Food Process...,Household
4,4,Secret Wish Women's Navy-Blue Towel Bathrobe (...,Household


In [ ]:
data02['text'] = data02['text'].astype(str)

Lakukan pre-processing untuk mendapatkan set token yang sudah berada dalam
bentuk dasar sesuai standard tata bahasa. Pada kode dibawah ini adalah fungsi untuk melakukan pre-processing dengan case folding, menghilangkan angka, menghilangkan tanda baca, dan menghilangkan spasi berlebih. Selanjutnya teks akan ditokenisasi menjadi kata-kata. Kemudian, kita akan menghilangkan stop word dalam token tersebut. Lalu, lakukan lematisasi kata dari token yang diperoleh. Terakhir, gabungkan kembali token menjadi kalimat, sehingga kita dapatkan kalimat yang sudah dibersihkan

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Case folding
    text = text.lower()

    # Remove number
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove multiple space
    text = re.sub(r'\s+', ' ', text)

    # Tokenisasi teks menjadi kata-kata
    tokens = word_tokenize(text)

    # Hapus stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Lematisasi kata
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Gabungkan kembali kata-kata menjadi kalimat
    processed_text = ' '.join(tokens)

    return processed_text

In [ ]:
data02['text'] = data02['text'].apply(preprocess_text)
print(data02['text'])

0        theory everything review stephen hawking theor...
1        computer network top approach author behrouz f...
2        sajani premium quality brown wooden coat hange...
3        bosch lifestyle mcmm watt food processor black...
4        secret wish woman navyblue towel bathrobe free...
                               ...                        
12601    lotus makeup ecostay insta smooth perfecting p...
12602    subtle art giving fck review resilience happin...
12603    elevanto premium collection th sleeve terry co...
12604    wd passport tb portable external hard drive ye...
12605    storite foot mm male mm female jack extension ...
Name: text, Length: 12606, dtype: object


Output diatas merupakan text dari data02 yang sudah di pre-process dan siap dipakai

Model LLM yang saya gunakan untuk menyelesaikan kasus ini adalah BERT (Bidirectional Encoder Representations from Transformers). BERT adalah salah satu model yang sangat populer dan efektif dalam pemrosesan bahasa alami (Natural Language Processing/NLP) dan pemahaman teks. Modeling dengan BERT (Bidirectional Encoder Representations from Transformers) menggunakan arsitektur transformer untuk mempelajari representasi kata yang kontekstual. BERT dapat memahami hubungan kata dalam kalimat dengan melibatkan konteks sebelum dan sesudahnya. Keunggulan BERT terletak pada representasi yang lebih baik dan kemampuan fine-tuning untuk tugas klasifikasi khusus.

Pertama-tama define tokenizer yang digunakan yaitu BERT Tokenizer yang sudah di pre-trained sebelumnya. Selain itu, define juga label yang akan digunakan.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'Household':0,
          'Books':1,
          'Electronics':2,
          'Clothing & Accessories':3,
          }

Selanjutnya, buat fungsi `Dataset` untuk membantu dalam mengatur dan memfasilitasi akses data dalam batch, baik untuk teks maupun label.

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

Selanjutnya split data ke dalam train, test dan validation dengan proporsi 80% data train, 10% data test, dan 10% data valid.

In [ ]:
np.random.seed(99)
df_train, df_val, df_test = np.split(data02.sample(frac=1, random_state=42), [int(.8*len(data02)), int(.9*len(data02))])

print(len(df_train),len(df_val), len(df_test))

10084 1261 1261


Buat model BERT dari hasil pretrained dengan nilai dropout 0.5 dimana inputnya 768 dan output yang diharapkan 4 untuk mengklasifikasikan keempat kategori yaitu Books, Clothing & Accessories, Electronics, dan Household

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

Disini saya melakukan training dengan melakukan tuning pada beberapa parameter seperti :
* `batch_size` = Batch size yang besar dapat mempercepat proses training karena lebih banyak sampel yang diproses secara paralel dalam satu iterasi. Namun penggunaan batch size yang lebih kecil dapat memperbarui bobot lebih sering, dan ini dapat membantu dalam memperbaiki model yang kompleks atau dalam menghadapi dataset yang rumit. Disini saya menggunakan 16 jumlah batch size, dimana angka ini cukup besar untuk mempercepat proses training, namun juga cukup kecil untuk mengoptimalkan hasil training model. Jika Anda ingin mendapatkan hasil training model yang lebih optimal sangat disarankan untuk menurunkan jumlah batch, namun perlu diketahui bahwa ukuran batch yang lebih besar atau lebih kecil dapat memberikan performa yang baik tergantung pada dataset dan tugas yang Anda hadapi.
* `epoch` = Dengan menggunakan jumlah epoch yang lebih besar, model memiliki lebih banyak kesempatan untuk melihat dan mempelajari pola pada data pelatihan. Ini dapat membantu model untuk mengoptimalkan parameter dan meningkatkan kinerjanya seiring berjalannya waktu. Pada kesempatan kali ini saya menggunakan 5 epoch untuk training. Jika Anda ingin mendapatkan hasil training model yang lebih optimal sangat disarankan untuk menambah jumlah epoch.
* `learning_rate` = Learning rate mengontrol seberapa besar langkah yang diambil pada setiap pembaruan parameter model selama pelatihan. Jika learning rate terlalu besar, langkah pembaruan parameter dapat menjadi terlalu besar yang menyebabkan model melompati minimum lokal atau bahkan bergerak menjauhinya. Hasilnya, model mungkin tidak konvergen atau memiliki performa yang buruk. Namun, jika learning rate terlalu kecil, langkah pembaruan parameter dapat menjadi terlalu kecil dan memperlambat proses pelatihan. Pada proses training ini saya menggunakan 1e-6 sebagai learning ratenya, dimana angka ini merupakan angka yang biasanya digunakan atau disarankan untuk melakukan training model.

Selain itu, disini saya menggunakan Adam Optimizer dan Cross Entropy sebagai metode untuk menghitung loss nya.

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 5
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 631/631 [14:20<00:00,  1.36s/it]


Epochs: 1 | Train Loss:  0.075                 | Train Accuracy:  0.555                 | Val Loss:  0.045                 | Val Accuracy:  0.817


100%|██████████| 631/631 [14:23<00:00,  1.37s/it]


Epochs: 2 | Train Loss:  0.033                 | Train Accuracy:  0.881                 | Val Loss:  0.024                 | Val Accuracy:  0.938


100%|██████████| 631/631 [14:24<00:00,  1.37s/it]


Epochs: 3 | Train Loss:  0.020                 | Train Accuracy:  0.936                 | Val Loss:  0.017                 | Val Accuracy:  0.948


100%|██████████| 631/631 [14:23<00:00,  1.37s/it]


Epochs: 4 | Train Loss:  0.015                 | Train Accuracy:  0.951                 | Val Loss:  0.015                 | Val Accuracy:  0.949


100%|██████████| 631/631 [14:24<00:00,  1.37s/it]


Epochs: 5 | Train Loss:  0.012                 | Train Accuracy:  0.960                 | Val Loss:  0.014                 | Val Accuracy:  0.949


Output diatas merupakan hasil akurasi dan loss dari train dan valid pada setiap epoch. Jika diperhatikan, dapat dilihat bahwa loss kian menurun pada setiap epoch, dan akurasi kian meningkat pada setiap epoch dan mencapai 96% train accuracy pada epoch ke 5 serta mencapai 95% val accuracy pada epoch ke 5. Hal ini menunjukkan bahwa model berhasil mempelajari data dengan sangat baik.

Terakhir, lakukan evaluasi model untuk melihat sebarapa baik model LLM tersebut melakukan klasifikasi terhadap text yang diberikan

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)

Test Accuracy:  0.940


Berdasarkan output diatas, terlihat bahwa test akurasinya menunjukkan angka sekitar 94%, ini artinya model BERT berhasil memahami dan mempelajari hubungan kata dalam kalimat yang melibatkan konteks sebelum dan sesudahnya. Hal ini terbukti dari kemampuan BERT yang luar biasa dalam melakukan proses klasifikasi label berdasarkan deskripsi produk, sehingga memberikan hasil prediksi yang baik.

## Nomor 3

Import Library yang dibutuhkan untuk nomor 3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import nltk
import re
import gensim
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim import corpora, models

Import data yang digunakan untuk menyelesaikan kasus nomor 3

In [ ]:
data03 = pd.read_csv('/content/drive/MyDrive/UAS Text Mining/data_3A.csv')

In [ ]:
data03.head()

,Unnamed: 0,ABSTRACT
0,0,Predictive models allow subject-specific inf...
1,1,Rotation invariance and translation invarian...
2,2,We introduce and develop the notion of spher...
3,3,The stochastic Landau--Lifshitz--Gilbert (LL...
4,4,Fourier-transform infra-red (FTIR) spectra o...


In [ ]:
data03['ABSTRACT'] = data03['ABSTRACT'].astype(str)


Saya menggunakan pendekatan Latent Dirichlet Allocation (LDA) dalam pemodelan data teks ini karena pendekatan ini tidak memerlukan label untuk mengelompokkan topik-topik dalam teks. LDA adalah metode yang digunakan untuk mengidentifikasi topik-topik yang tersembunyi dalam kumpulan dokumen tanpa adanya anotasi label. Dengan LDA, kita dapat mengungkap pola tematik yang mendasari teks dan mengorganisasikan dokumen ke dalam kelompok topik yang berkaitan. Pendekatan ini sangat berguna ketika kita ingin memahami struktur tematik dalam dataset teks tanpa memiliki informasi label yang jelas. Dengan menggunakan LDA, kita dapat mengidentifikasi topik-topik yang ada dalam data dan menghasilkan pemahaman yang lebih dalam tentang isi dan konten teks tersebut.

Lakukan pre-processing untuk mendapatkan set token yang sudah berada dalam
bentuk dasar sesuai standard tata bahasa. Pada kode dibawah ini adalah fungsi untuk melakukan pre-processing dengan case folding, menghilangkan angka, menghilangkan tanda baca, dan menghilangkan spasi berlebih. Selanjutnya teks akan ditokenisasi menjadi kata-kata. Kemudian, kita akan menghilangkan stop word dalam token tersebut. Lalu, lakukan lematisasi kata dari token yang diperoleh. Terakhir, buat fungsi baru untuk menggabungkan kembali token menjadi kalimat, sehingga kita dapatkan kalimat yang sudah dibersihkan

In [ ]:
stop_words = set(stopwords.words('english'))

def tokenize_text(text):
    # Case folding
    text = text.lower()

    # Remove number
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove multiple space
    text = re.sub(r'\s+', ' ', text)

    # Tokenisasi teks menjadi kata-kata
    tokens = word_tokenize(text)

    # Hapus stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Lematisasi kata
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

def cleaned(text):
    tokens = tokenize_text(text)
    processed_text = ' '.join(tokens)

    return processed_text

Lakukan tokenisasi untuk mendapatkan token dari abstract yang diberikan

In [ ]:
tokens = data03['ABSTRACT'].apply(tokenize_text)
print(tokens)

0     [predictive, model, allow, subjectspecific, in...
1     [rotation, invariance, translation, invariance...
2     [introduce, develop, notion, spherical, polyha...
3     [stochastic, landaulifshitzgilbert, llg, equat...
4     [fouriertransform, infrared, ftir, spectrum, s...
                            ...                        
95    [paper, presented, novel, convolutional, neura...
96    [variety, representation, learning, approach, ...
97    [motivated, perelmans, pseudo, locality, theor...
98    [bound, exponential, sum, appears, study, irre...
99    [investigate, effect, dimensional, crossover, ...
Name: ABSTRACT, Length: 100, dtype: object


Selanjutkan, siapkan juga cleaned text yang akan digunakan sebagai sampel untuk melakukan prediksi topik dari text yang sudah dibersihkan

In [ ]:
cleaned_text = data03['ABSTRACT'].apply(cleaned)
print(cleaned_text)

0     predictive model allow subjectspecific inferen...
1     rotation invariance translation invariance gre...
2     introduce develop notion spherical polyharmoni...
3     stochastic landaulifshitzgilbert llg equation ...
4     fouriertransform infrared ftir spectrum sample...
                            ...                        
95    paper presented novel convolutional neural net...
96    variety representation learning approach inves...
97    motivated perelmans pseudo locality theorem ri...
98    bound exponential sum appears study irregulari...
99    investigate effect dimensional crossover groun...
Name: ABSTRACT, Length: 100, dtype: object


Selanjutnya, buat fungsi `build_dic` untuk membuat objek kamus baru menggunakan kelas Dictionary dari library Gensim dengan beberapa ketentuan seperti :
* `no_below=10` berarti kata-kata yang muncul kurang dari 10 kali dalam keseluruhan teks akan dihapus dari kamus.
* `no_above=0.2` berarti kata-kata yang muncul dalam lebih dari 20% teks akan dihapus dari kamus.
* `keep_n=100000` berarti kamus akan mempertahankan 100.000 kata teratas berdasarkan frekuensinya.

In [ ]:
def build_dic(text):
    dictionary = Dictionary(text)
    dictionary.filter_extremes(no_below=10, no_above=0.2, keep_n= 100000)
    return dictionary

Lalu, buat fungsi `build_vec` untuk membangun representasi vektor dari teks menggunakan model Bag-of-Words (BOW) berdasarkan kamus yang diberikan.

In [ ]:
def build_vec(text,dictionary):
    bow_corpus = [dictionary.doc2bow(doc) for doc in text]
    return bow_corpus

Berikutnya, buat fungsi `vector_tfidf` untuk menghitung representasi vektor TF-IDF (Term Frequency-Inverse Document Frequency) dari representasi vektor Bag-of-Words (BOW) yang diberikan

In [ ]:
def vector_tfidf(bow_corpus):
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    return corpus_tfidf

Buat juga fungsi `model_lda` untuk melatih model Latent Dirichlet Allocation (LDA) menggunakan representasi vektor Bag-of-Words (BoW) dan mengembalikan model LDA yang telah dilatih.

In [ ]:
def model_lda(dictionary,bow_corpus,num_topic,alpha,eta):
    lda_model =  gensim.models.LdaMulticore(bow_corpus,
                                   num_topics = num_topic,
                                   id2word = dictionary,
                                   passes = 25,
                                   workers = 5,
                                   alpha=alpha,
                                   eta=eta)
    return lda_model

Terakhir, buat fungsi `score_perf` untuk menghitung coherence score dari model LDA menggunakan teks dan kamus yang diberikan.

In [ ]:
def score_perf(lda_model,text,dictionary):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

Setelah membuat semua fungsi yang dibutuhkan, lanjutkan dengan membuat dictionary dari token yang sudah dicleansing, lalu buat juga BOW Corpusnya, serta TF-IDF dari corpus yang sudah dibuat.

In [ ]:
dictionary = build_dic(tokens)

In [ ]:
bow_corpus = build_vec(tokens, dictionary)

In [ ]:
corpus_tfidf = vector_tfidf(bow_corpus)

### Clustering 3 Topic

Kita akan melakukan LDA dengan beberapa jumlah topik, pertama-tama saya akan menggunakan 3 jumlah topik untuk melakukan clustering terlebih dahulu.

In [ ]:
lda_model1 = model_lda(dictionary, corpus_tfidf, 3, 0.7, 0.7)

In [ ]:
coherence_score=score_perf(lda_model1, tokens, dictionary)
print(coherence_score)

0.2465261092173021


Dapat dilihat bahwa nilai coherence score dari model1 menunjukkan angka 0.2465, dimana angka ini dapat terbilang cukup rendah. Artinya kata-kata yang diatribusikan ke dalam topik-topik tersebut tidak memiliki hubungan yang kuat atau tematik yang jelas

In [ ]:
for idx, topic in lda_model1.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.048*"network" + 0.046*"system" + 0.038*"approach" + 0.037*"learning" + 0.032*"framework" + 0.031*"performance" + 0.031*"task" + 0.028*"application" + 0.028*"function" + 0.026*"stateoftheart"


Topic: 1 
Words: 0.042*"measure" + 0.039*"phase" + 0.034*"used" + 0.031*"present" + 0.029*"function" + 0.028*"proposed" + 0.028*"parameter" + 0.027*"well" + 0.026*"two" + 0.025*"main"


Topic: 2 
Words: 0.037*"dynamic" + 0.037*"energy" + 0.036*"state" + 0.034*"finite" + 0.033*"equation" + 0.030*"domain" + 0.027*"condition" + 0.026*"solution" + 0.026*"first" + 0.026*"data"




Output diatas merupakan informasi mengenai setiap topik yang dihasilkan oleh model LDA. Informasi ini termasuk nomor topik dan kata-kata yang paling relevan dengan topik tersebut. Misalnya pada topic 0, kata yang paling relevan dengan topic tersebut adalah `network` dengan probabilitas sebesar 0.048. Pada topic 1, kata yang paling relevan dengan topic tersebut adalah `measure` dengan probabilitas sebesar 0.042. Pada topic 2, kata yang paling relevan dengan topic tersebut adalah `dynamic` dengan probabilitas sebesar 0.037.

Lakukan prediksi topic clustering pada sampel. Disini saya mengambil 5 sampel, yang saya ambil dari data ke 55 sampai data ke 60

In [ ]:
X_test_token = tokens.iloc[55:60]
X_test_cleaned = cleaned_text.iloc[55:60]

In [ ]:
X_test_doc = [i for i in X_test_cleaned]

In [ ]:
bow_corpusTest = build_vec(X_test_token, dictionary)

In [ ]:
corpus_tfidfTest = vector_tfidf(bow_corpusTest)

Lakukan prediksi topik clustering menggunakan model LDA yang telah dibuat sebelumnya. Lalu tampilkan juga 2 urutan tertinggi dari score topik yang paling cocok dengan dokumen tersebut.

In [ ]:
#Top 2 The highest score of topic
test_shape=len(X_test_doc)
topic1=[]
score1=[]
topic2=[]
score2=[]
sentence=[]
for doc in range(test_shape):
    sentence.append(X_test_doc[doc])
    topic1.append(sorted(lda_model1[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[0][0])
    score1.append(sorted(lda_model1[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[0][1])
    topic2.append(sorted(lda_model1[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[1][0])
    score2.append(sorted(lda_model1[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[1][1])

In [ ]:
sentence=pd.DataFrame(sentence,columns=['sentence']).reset_index()
topic1=pd.DataFrame(topic1,columns=['topic1']).reset_index()
score1=pd.DataFrame(score1,columns=['score1']).reset_index()
topic2=pd.DataFrame(topic2,columns=['topic2']).reset_index()
score2=pd.DataFrame(score2,columns=['score2']).reset_index()
test_result=pd.concat([sentence,topic1,score1,topic2,score2],axis=1)

In [ ]:
test_result1 = test_result[['sentence','topic1','score1','topic2','score2']]
test_result1.head()

,sentence,topic1,score1,topic2,score2
0,investigating emergence particular cell type r...,2,0.771570,0,0.129495
1,stimuliresponsive material modify shape respon...,2,0.508125,1,0.285351
2,today landscape robotics dominated vertical in...,0,0.580442,1,0.300354
3,machine learning model especially based deep a...,0,0.833165,1,0.128918
4,study query complexity cake cutting give lower...,0,0.333333,1,0.333333


Berdasarkan output diatas, dapat dilihat bahwa data pertama teridentifikasi sebagai topik 2 sebesar 77,16%, dan juga teridentifikasi sebagai topik 0 sebesar 12,95%, sehingga dapat disimpulkan bahwa data pertama lebih cocok untuk masuk ke dalam topik 2. Begitu juga data yang lainnya. Namun, data terakhir teridentifikasi sebagai topik 0 sebesar 33,33%, dan juga teridentifikasi sebagai topik 1 sebesar 33,33%, sehingga data terakhir dapat termasuk ke dalam topik 0, namun juga dapat termasuk ke dalam topik 1 karena memiliki tingkat probabilitas yang sama.

### Clustering 5 Topic

Selanjutnya, kita akan mencoba melakukan LDA dengan menggunakan 5 jumlah topik untuk melakukan clustering.

In [ ]:
lda_model2 = model_lda(dictionary, corpus_tfidf, 5, 0.7, 0.7)

In [ ]:
coherence_score=score_perf(lda_model2, tokens, dictionary)
print(coherence_score)

0.22450146008606514


Dapat dilihat bahwa nilai coherence score dari model2 menunjukkan angka 0.2245, dimana angka ini dapat terbilang cukup rendah. Artinya kata-kata yang diatribusikan ke dalam topik-topik tersebut tidak memiliki hubungan yang kuat atau tematik yang jelas. Angka ini juga lebih kecil dibandingkan dengan nilai coherence score dari model1, yang artinya proses clustering akan memberikan hasil yang lebih baik jika menggunakan 3 topik.

In [ ]:
for idx, topic in lda_model2.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.028*"data" + 0.026*"sample" + 0.023*"performance" + 0.022*"used" + 0.021*"network" + 0.021*"learning" + 0.021*"task" + 0.020*"system" + 0.020*"approach" + 0.019*"problem"


Topic: 1 
Words: 0.033*"system" + 0.022*"approach" + 0.021*"used" + 0.020*"solution" + 0.020*"data" + 0.019*"network" + 0.019*"dynamic" + 0.019*"present" + 0.019*"equation" + 0.019*"set"


Topic: 2 
Words: 0.021*"dynamic" + 0.021*"algorithm" + 0.021*"effect" + 0.020*"state" + 0.020*"finite" + 0.020*"data" + 0.019*"local" + 0.019*"approach" + 0.019*"condition" + 0.019*"problem"


Topic: 3 
Words: 0.046*"function" + 0.030*"network" + 0.023*"measure" + 0.019*"system" + 0.018*"state" + 0.018*"present" + 0.018*"learning" + 0.018*"problem" + 0.018*"two" + 0.017*"approach"


Topic: 4 
Words: 0.026*"phase" + 0.022*"approach" + 0.022*"system" + 0.021*"domain" + 0.021*"energy" + 0.020*"finite" + 0.020*"set" + 0.019*"performance" + 0.019*"state" + 0.019*"measure"




Output diatas merupakan informasi mengenai setiap topik yang dihasilkan oleh model LDA. Informasi ini termasuk nomor topik dan kata-kata yang paling relevan dengan topik tersebut. Misalnya pada topic 0, kata yang paling relevan dengan topic tersebut adalah `data` dengan probabilitas sebesar 0.028. Pada topic 1, kata yang paling relevan dengan topic tersebut adalah `system` dengan probabilitas sebesar 0.033. Pada topic 2, kata yang paling relevan dengan topic tersebut adalah `dynamic` dengan probabilitas sebesar 0.021. Pada topic 3, kata yang paling relevan dengan topic tersebut adalah `function` dengan probabilitas sebesar 0.046. Pada topic 4, kata yang paling relevan dengan topic tersebut adalah `phase` dengan probabilitas sebesar 0.026.

Dengan menggunakan data testing yang sama seperti sebelumnya, lakukan prediksi topik clustering menggunakan model LDA yang telah dibuat sebelumnya. Lalu tampilkan juga 2 urutan tertinggi dari score topik yang paling cocok dengan dokumen tersebut.

In [ ]:
# Top 2 The highest score of topic
test_shape=len(X_test_doc)
topic1=[]
score1=[]
topic2=[]
score2=[]
sentence=[]
for doc in range(test_shape):
    sentence.append(X_test_doc[doc])
    topic1.append(sorted(lda_model2[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[0][0])
    score1.append(sorted(lda_model2[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[0][1])
    topic2.append(sorted(lda_model2[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[1][0])
    score2.append(sorted(lda_model2[bow_corpusTest[doc]], key=lambda tup: -1*tup[1])[1][1])

In [ ]:
sentence=pd.DataFrame(sentence,columns=['sentence']).reset_index()
topic1=pd.DataFrame(topic1,columns=['topic1']).reset_index()
score1=pd.DataFrame(score1,columns=['score1']).reset_index()
topic2=pd.DataFrame(topic2,columns=['topic2']).reset_index()
score2=pd.DataFrame(score2,columns=['score2']).reset_index()
test_result=pd.concat([sentence,topic1,score1,topic2,score2],axis=1)

In [ ]:
test_result2 = test_result[['sentence','topic1','score1','topic2','score2']]
test_result2.head()

,sentence,topic1,score1,topic2,score2
0,investigating emergence particular cell type r...,2,0.406604,4,0.180537
1,stimuliresponsive material modify shape respon...,3,0.282587,2,0.253033
2,today landscape robotics dominated vertical in...,1,0.247078,0,0.207291
3,machine learning model especially based deep a...,0,0.796006,3,0.053406
4,study query complexity cake cutting give lower...,0,0.200000,1,0.200000


Berdasarkan output diatas, dapat dilihat bahwa data pertama teridentifikasi sebagai topik 2 sebesar 40,66%, dan juga teridentifikasi sebagai topik 4 sebesar 18,05%, sehingga dapat disimpulkan bahwa data pertama lebih cocok untuk masuk ke dalam topik 2. Begitu juga data yang lainnya. Namun, data terakhir teridentifikasi sebagai topik 0 sebesar 20%, dan juga teridentifikasi sebagai topik 1 sebesar 20%, sehingga data terakhir dapat termasuk ke dalam topik 0, namun juga dapat termasuk ke dalam topik 1 karena memiliki tingkat probabilitas yang sama.

## Nomor 4

Import Library yang dibutuhkan untuk nomor 4

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

Pertama-tama, buat fungsi `read_article` untuk membaca teks dari file dan memisahkannya menjadi kalimat-kalimat terpisah berdasarkan pemisah titik (".")

In [ ]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    # Split the sentences based on dot (.) separator
    article = filedata[0].split(". ")
    sentences = []
    for i, sentence in enumerate(article):
        print(f"{i+1}. {sentence}")
        # Remove non-alphabetic characters
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()

    return sentences

Lalu, buat fungsi `sentence_similarity` untuk menghitung tingkat kesamaan antara dua kalimat menggunakan pendekatan representasi vektor

In [ ]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    #build vector 0 with dimension following the len of all_words
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

Berikutnya, buat fungsi `build_similarity_matrix` untuk membangun matriks kesamaan antara kalimat-kalimat dalam bentuk array dua dimensi.

In [ ]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

Terakhir, buat fungsi `generate_summary` untuk menghasilkan ringkasan teks dari file teks yang diberikan dengan pendekatan extractive serta menampilkan Top 3 Ranked Sentences berdasarkan tingkat similiraty matrix yang dihitung menggunakan fungsi yang sudah dibuat sebelumnya

In [ ]:
def generate_summary(file_name, top_n):
    stop_words = stopwords.words('english')
    summarize_text = []

    print("Sentences :")
    # Step 1 - Read text and split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similarity Matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    print("\nIndexes of top ranked sentences:")
    for i, (scores, sentence) in enumerate(ranked_sentences):
        print(f"{i+1}. Score: {scores:.4f} | Sentence: {sentence}")

        if i >= top_n - 1:
            break

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentences[i][1]))

    # Step 5 - Output the summarized text
    print("\nSummarized Text:")
    for i, summary in enumerate(summarize_text):
        print(f"{i+1}. {summary}")

In [ ]:
generate_summary('/content/drive/MyDrive/UAS Text Mining/data_4A.txt', 3)

Sentences :
1. ﻿Stem cells are cells that (a) on dividing are capable of recreating themselves in at least one daughter cell and (b) have the capacity to differentiate into several linages
2. Stem cells play a critical and essential role in the human body not only by providing the starting material for organs and tissues but also for their continual maintenance, growth, and renewal throughout ontogeny [e.g
3. hematopoietic stem cells (HSCs) → erythrocytes, neural stem cells (NSCs) → neurons, etc]
4. As the embryo and the fetus develop, stem cells are seeded into the various tissues and organs where they remain throughout life (Figure 1)
5. Stem cells can also exist temporarily during embryonic development and can be artificially established ex vivo from transient stages of differentiation
6. Embryonic stem (ES) cells of mice, for example, are isolated by culturing transiently existing inner cell mass cells of early embryos
7. ES cells are cells that retain the properties of primitive e

Output diatas merupakan 3 kalimat terpenting yang dapat merepresentasikan keseluruhan teks beserta dengan nilai similarity matrixnya